In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/15 12:43:04 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.176.27 instead (on interface en0)
22/09/15 12:43:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 12:43:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/15 12:43:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/15 12:43:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
census_04A_age = spark.read.parquet('../data/curated/census_data_2021_04A.parquet')
census_04B_age = spark.read.parquet('../data/curated/census_data_2021_04B.parquet')
census_17A_income = spark.read.parquet('../data/curated/census_data_2021_17A.parquet')
census_17B_income = spark.read.parquet('../data/curated/census_data_2021_17B.parquet')
census_17C_income = spark.read.parquet('../data/curated/census_data_2021_17C.parquet')
correspondences = spark.read.parquet('../data/curated/correspondences.parquet')
postcode = spark.read.parquet('../data/curated/postcode_SA2.parquet/')


In [3]:
#Merge the age dataframe
Age_Data = census_04A_age.join(census_04B_age, census_04A_age.SA2_CODE_2021 == census_04B_age.SA2_CODE_2021).drop(census_04B_age.SA2_CODE_2021)

In [4]:
#Select the column in the age dataframe (18-65) and calculate the sum
Age_Data_select = Age_Data.select(Age_Data.columns[0::18])
Age_Data_total = Age_Data_select.withColumn("result", sum([col(x) for x in Age_Data_select.columns[5:14]]))
Age_Data_total.limit(3)


SA2_CODE_2021,Age_yr_0_4_P,Age_yr_5_9_P,Age_yr_10_14_P,Age_yr_15_19_P,Age_yr_20_24_P,Age_yr_25_29_P,Age_yr_30_34_P,Age_yr_35_39_P,Age_yr_40_44_P,Age_yr_45_49_P,Age_yr_50_54_P,Age_yr_55_59_P,Age_yr_60_64_P,Age_yr_65_69_P,Age_yr_70_74_P,Age_yr_75_79_P,Tot_P,result
101021007,207,254,229,173,122,172,192,246,202,291,351,353,435,338,302,249,4343,2364.0
101021008,526,530,566,490,515,629,637,588,546,556,594,580,513,429,325,227,8517,5158.0
101021009,669,587,482,433,689,1130,1238,903,740,705,690,674,587,509,418,286,11342,7356.0


In [5]:
Age_Data_total = Age_Data_total.select("SA2_CODE_2021","result","Tot_P")
Age_Data_total.limit(3)

SA2_CODE_2021,result,Tot_P
101021007,2364.0,4343
101021008,5158.0,8517
101021009,7356.0,11342


In [6]:
Age_Data_rate = Age_Data_total.withColumn("age_percentage", Age_Data_total.result /Age_Data_total.Tot_P)
Age_Data_rate = Age_Data_rate.select("SA2_CODE_2021","age_percentage")
Age_Data_rate.limit(3)

SA2_CODE_2021,age_percentage
101021007,0.5443241998618467
101021008,0.6056123048021604
101021009,0.6485628636924705


In [7]:
income = census_17A_income.join(census_17B_income, census_17A_income.SA2_CODE_2021 == census_17B_income.SA2_CODE_2021).drop(census_17B_income.SA2_CODE_2021)
income_data = income.join(census_17C_income, income.SA2_CODE_2021 == census_17C_income.SA2_CODE_2021).drop(census_17C_income.SA2_CODE_2021)

income_data_del = income_data.select(income_data.columns[0::10])
income_data_select =income_data_del.withColumn("result", sum([col(x) for x in income_data_del.columns[44:50]]))
income_data_select.limit(3)

22/09/15 12:43:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2_CODE_2021,M_Neg_Nil_income_Tot,M_1_149_Tot,M_150_299_Tot,M_300_399_Tot,M_400_499_Tot,M_500_649_Tot,M_650_799_Tot,M_800_999_Tot,M_1000_1249_Tot,M_1250_1499_Tot,M_1500_1749_Tot,M_1750_1999_Tot,M_2000_2999_Tot,M_3000_3499_Tot,M_3500_more_Tot,M_PI_NS_ns_Tot,M_Tot_Tot,F_Neg_Nil_income_Tot,F_1_149_Tot,F_150_299_Tot,F_300_399_Tot,F_400_499_Tot,F_500_649_Tot,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_3499_Tot,F_3500_more_Tot,F_PI_NS_ns_Tot,F_Tot_Tot,P_Neg_Nil_income_Tot,P_1_149_Tot,P_150_299_Tot,P_300_399_Tot,P_400_499_Tot,P_500_649_Tot,P_650_799_Tot,P_800_999_Tot,P_1000_1249_Tot,P_1250_1499_Tot,P_1500_1749_Tot,P_1750_1999_Tot,P_2000_2999_Tot,P_3000_3499_Tot,P_3500_more_Tot,P_PI_NS_ns_Tot,P_Tot_Tot,result
101021007,90,52,96,146,141,114,120,120,185,120,113,94,162,42,68,184,1840,136,88,97,160,174,160,141,146,135,101,83,52,114,12,32,170,1807,225,140,196,307,312,274,264,266,323,217,192,146,277,57,98,355,3650,987.0
101021008,226,56,116,204,177,150,194,265,377,312,322,227,361,59,93,246,3388,274,113,175,234,306,261,238,284,318,289,271,172,259,45,29,245,3509,495,165,290,435,484,416,435,549,697,599,592,404,619,102,125,492,6895,2441.0
101021009,222,90,132,248,276,266,317,469,611,487,368,290,542,83,110,321,4832,319,98,145,336,437,336,371,425,468,383,368,254,393,35,69,330,4771,541,190,280,588,711,600,694,896,1080,866,738,545,939,117,179,643,9608,3384.0


In [8]:
income_Data_total = income_data_select.select("SA2_CODE_2021","result","P_Tot_Tot")
income_Data_total.limit(3)

SA2_CODE_2021,result,P_Tot_Tot
101021007,987.0,3650
101021008,2441.0,6895
101021009,3384.0,9608


In [9]:
income_Data_total = income_Data_total.withColumn("income_percentage", income_Data_total.result / income_Data_total.P_Tot_Tot)
income_Data_total = income_Data_total.select("SA2_CODE_2021","income_percentage")

In [11]:
census_data = income_Data_total.join(Age_Data_rate, income_Data_total.SA2_CODE_2021 == Age_Data_rate.SA2_CODE_2021).drop(Age_Data_rate.SA2_CODE_2021)
census_data.write.parquet('../data/curated/final_census.parquet')
